In [1]:
import requests
import pandas as pd
import numpy as np
import time

In [2]:
url = 'https://api.pushshift.io/reddit/search/submission'

## Run this first to get the last utc time for the first run

In [3]:
params = {
    'subreddit': 'Austin',
    'size': 100    
}

res = requests.get(url, params)

res.status_code

200

In [4]:
res.json().keys()

dict_keys(['data'])

In [5]:
posts = res.json()['data']

In [6]:
posts[0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'lenagracep',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_4euo58mu',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1630957968,
 'domain': 'reddit.com',
 'full_link': 'https://www.reddit.com/r/Austin/comments/pj7267/the_parking_lot_at_upper_bull_creek_this_morning/',
 'gallery_data': {'items': [{'caption': 'Why? This was my first time out at upper bull creek. Trash EVERYWHERE. Glass bottles, beer cans, bottle caps, beer boxes, styrofoam plates…',
    'id': 69734600,
    'media_id': 'x14nzuxttxl71'},
   {'caption': 'All over the parking lot ',
    'id': 69734601,
    'media_id': 'l6p6ctxttxl71'},
   {'caption': 'And more plastic…',
    'id': 69734602,
    'media_id': '9iddtuxttxl71'},
   {'id': 69734603, '

In [7]:
df = pd.DataFrame(posts)[['subreddit', 'title', 'selftext', 'created_utc']]
df.tail()

,subreddit,title,selftext,created_utc
95,Austin,I got this Bitcoin investment company that I h...,[removed],1630859232
96,Austin,Am I missing a joke here?,What's with the tigers on the [Barton Creek en...,1630859191
97,Austin,College advice; getting into UT?,Any tips/advice for college admissions and get...,1630858905
98,Austin,Neighbor stealing packages,Hello all. I need some advice. My downstairs n...,1630858173
99,Austin,Looking for someone!,Anyone know or ever play racquetball with a ma...,1630857912


## Now run the scraper function

In [9]:
#Will scrape redit/Austin for n times and return a df
#will need to run it one time outside of function to get the laststatus time and create the df
last_time = df['created_utc'].iloc[-1]
#1630796537

def scraped(n):
    df_final = df
    for i in range (n):
        
        params = {
        'subreddit': 'Austin',
        'size': 100,
        'before': df['created_utc'].iloc[-1]
        }
        res = requests.get(url, params)
        posts = res.json()['data']
        df2 = pd.DataFrame(posts)[['subreddit', 'title', 'selftext', 'created_utc']]
        df_final = pd.concat([df_final, df2])
        #need to update last_time
        #last_time = df['created_utc'].iloc[-1]
        time.sleep(10)
    
    return df_final


## Define number of iterations, remember you pull 100 post per iteration

In [10]:
#Call the function
df_Austin = scraped(120)

## Quick test of intergrity of data frame to make sure data aquisition is complete

In [11]:
df_Austin.head()

,subreddit,title,selftext,created_utc
0,Austin,The parking lot at upper bull creek this morni...,,1630957968
1,Austin,Happy birthday Sadie,,1630956095
2,Austin,I guess this accessible spot is strictly for t...,,1630955505
3,Austin,NEED HELP,[removed],1630954679
4,Austin,Lost black and white tuxedo cat around Hancock...,Hello! Lord Byron is a large boy (about 14 pou...,1630954248


In [12]:
df_Austin.shape

(12100, 4)

In [1]:
#df_Austin['selftext'].value_counts()

In [16]:
len(df_Austin.loc[df_Austin['selftext'] == '[removed]'])

491

## Remove blanks and [removed] content

In [17]:
df_Austin['selftext'].replace('', np.nan, inplace=True)

In [18]:
df_Austin['selftext'].replace('[removed]', np.nan, inplace=True)

In [2]:
#df_Austin.isnull().sum()

In [20]:
df_Austin.dropna(inplace = True)

In [21]:
df_Austin.shape

(6052, 4)

## Write to a csv

In [23]:
df_Austin.to_csv('data/df_Austin.csv', index=False)